In [0]:
from datetime import datetime
from zoneinfo import ZoneInfo
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_date, hour

In [0]:
# # comment later after passing the key value pairs as arguments
# tiers = ['bronze', 'silver', 'gold']
# adls_paths = {tier: f"abfss://{tier}@adlsnews.dfs.core.windows.net" for tier in tiers}
# bronze_path = adls_paths['bronze']
# silver_path = adls_paths['silver']
# gold_path = adls_paths['gold']
# seattle_tz = ZoneInfo("America/Los_Angeles")
# now_seattle = datetime.now(seattle_tz)
# current_date = now_seattle.strftime('%Y-%m-%d')
# bronze_write_path = f"{bronze_path}/{current_date}/latest_news.json"
# silver_write_path = f"{silver_path}/{current_date}"

In [0]:
# get the task values from the previous tasks
bronze_output= dbutils.jobs.taskValues.get(taskKey = "bronze", key = "bronze_output")
silver_output= dbutils.jobs.taskValues.get(taskKey = "silver", key = "silver_output")
silver_path = bronze_output.get("silver_path")
gold_path = bronze_output.get("gold_path")
current_date = bronze_output.get("current_date")
silver_write_path = silver_output.get("silver_write_path")

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-8098059504380056>, line 2
      1 # get the task values from the previous tasks
----> 2 bronze_output= dbutils.jobs.taskValues.get(taskKey = "bronze", key = "bronze_output")
      3 silver_output= dbutils.jobs.taskValues.get(taskKey = "silver", key = "silver_output")
      4 silver_path = bronze_output.get("silver_path")

File /databricks/python_shell/lib/dbruntime/dbutils.py:208, in DBUtils.JobsHandler.TaskValuesHandler.get(self, taskKey, key, default, debugValue)
    206 elif 'NotInJobContextException' in exceptionClassName:
    207     if debugValue is None:
--> 208         raise TypeError(
    209             'Must pass debugValue when calling get outside of a job context. debugValue cannot be None.'
    210         ) from None
    211     return debugValue
    213 raise

TypeError: Must pass debugValue when calling ge

In [0]:
# read the data from the silver adls
news = spark.read\
    .format("parquet")\
        .load(silver_write_path)
news.display()

author,category,country,description,image,language,published_at,source,title,url
Justin Lynch,sports,us,Connor Bedard commits to Chicago,https://library.sportingnews.com/styles/crop_style_16_9_desktop_webp/s3/2024-06/sporting-news-2022-sn-live-e97a2913-4adb-4215-b12f-50eaa9540b4c.jpeg.webp?itok=bAuiEHxe,en,2025-06-25T23:59:18Z,Sporting News,Hawks superstar makes firm statement on his future,https://www.sportingnews.com/us/nhl/news/hawks-superstar-makes-firm-statement-his-future/1988b92c5cfc7a1383a53153
Associated Press,general,us,"When Northwest Orient Flight 2501 crashed, it was the worst aviation disaster in U.S. history.",https://nypost.com/wp-content%2Fuploads%2Fsites%2F2%2F2025%2F06%2FUntitled-1_0318fd.jpg?quality%3D90%26strip%3Dall,en,2025-06-25T23:59:11Z,New York Post,"75 years after the then-worst crash in US history, search ends for a plane in Lake Michigan",https://nypost.com/2025/06/25/us-news/75-years-after-the-then-worst-crash-in-us-history-search-ends-for-a-plane-in-lake-michigan/
Caleigh Wells | AP,general,us,"People shopping for appliances often look for the Energy Star symbol. The U.S.-based program says its blue seal of approval on energy-efficient appliances saves households an average of $450 a year on their utility bills. Since its launch in 1992, Energy Star appliances also have helped prevent 4 billion metric tons of greenhouse gas emissions — the main cause of climate change.But the program’s future is unclear. The federal Environmental Protection Agency has announced an agency reorganization plan under which its Energy Star offices would be eliminated.That would make it more difficult...",null,en,2025-06-25T23:58:53Z,suntimes,How to find energy-efficient appliances if Energy Star program ends,https://chicago.suntimes.com/consumer-affairs/2025/06/25/energy-star-energy-efficient-appliances-program-alternatives-tips
Billy Heyen,sports,us,The new Boston defenseman could stick around.,https://library.sportingnews.com/styles/crop_style_16_9_desktop_webp/s3/2025-06/USATSI_25920677.jpg.webp?itok=eq8FuL87,en,2025-06-25T23:58:42Z,Sporting News,Bruins' Henri Jokiharju gets $10.8 million contract update,https://www.sportingnews.com/us/nhl/news/bruins-henri-jokiharju-108-million-contract-update/b7d54441b6fd3620af52ac14
Claire Fahy and Alyce McFadden,general,us,The state assemblyman ran a campaign tightly focused on issues related to affordability. Here is a look at where he stands on those issues and others.,https://static01.nyt.com/images/2025/06/25/multimedia/25nyc-election-mamdani-on-the-issues-topart-zblf/25nyc-election-mamdani-on-the-issues-topart-zblf-mediumSquareAt3X.jpg,en,2025-06-25T23:58:26Z,The New York Times,A Look at Zohran Mamdani’s Stances on Key Issues,https://www.nytimes.com/2025/06/25/nyregion/mamdani-policies-stances.html
Patrick Saunders,general,us,The lefty has succeeded at Coors Field but rarely dominated.,https://www.denverpost.com/wp-content/uploads/2017/04/clayton-kershaw.jpg?w=1400px&strip=all,en,2025-06-25T23:57:56Z,photos,"Dodgers’ Clayton Kershaw, eight strikeouts shy of 3,000, faces Rockies Thursday at Coors Field",https://www.denverpost.com/2025/06/25/clayton-kershaw-strikeouts-milestone-rockies-dodgers/
Óscar Contreras,general,us,"Lakewood police is asking for the public’s help to find two suspects accused of stealing approximately $15,000 worth of tools and equipment out of The Academy Place last month.",null,en,2025-06-25T23:56:53Z,thedenverchannel,Lakewood police seeking public’s help to identify theft suspects accused of stealing $15k worth of equipment,https://www.denver7.com/news/crime/lakewood-police-seeking-publics-help-to-identify-theft-suspects-accused-of-stealing-15k-worth-of-equipment
Investing.com,business,us,Lockheed Martin declares third quarter 2025 dividend of $3.30,https://i-invdn-com.investing.com/news/news_headline_rolled_108x81.jpg,en,2025-06-25T23:56:50Z,Investing.com | Stock Market Quotes & Financial News,Lockheed Martin declares third quarter 2025 dividend of 

In [0]:
# extract the date and hour from the published_at column
news = news.withColumn("published_date", to_date(col("published_at"))) \
           .withColumn("published_hour", hour(col("published_at")))

In [0]:
# fill missing values in the image column
news = news.fillna(value= "No URL Available", subset=["image"])
news.display()

author,category,country,description,image,language,published_at,source,title,url,published_date,published_hour
Justin Lynch,sports,us,Connor Bedard commits to Chicago,https://library.sportingnews.com/styles/crop_style_16_9_desktop_webp/s3/2024-06/sporting-news-2022-sn-live-e97a2913-4adb-4215-b12f-50eaa9540b4c.jpeg.webp?itok=bAuiEHxe,en,2025-06-25T23:59:18Z,Sporting News,Hawks superstar makes firm statement on his future,https://www.sportingnews.com/us/nhl/news/hawks-superstar-makes-firm-statement-his-future/1988b92c5cfc7a1383a53153,2025-06-25,23
Associated Press,general,us,"When Northwest Orient Flight 2501 crashed, it was the worst aviation disaster in U.S. history.",https://nypost.com/wp-content%2Fuploads%2Fsites%2F2%2F2025%2F06%2FUntitled-1_0318fd.jpg?quality%3D90%26strip%3Dall,en,2025-06-25T23:59:11Z,New York Post,"75 years after the then-worst crash in US history, search ends for a plane in Lake Michigan",https://nypost.com/2025/06/25/us-news/75-years-after-the-then-worst-crash-in-us-history-search-ends-for-a-plane-in-lake-michigan/,2025-06-25,23
Caleigh Wells | AP,general,us,"People shopping for appliances often look for the Energy Star symbol. The U.S.-based program says its blue seal of approval on energy-efficient appliances saves households an average of $450 a year on their utility bills. Since its launch in 1992, Energy Star appliances also have helped prevent 4 billion metric tons of greenhouse gas emissions — the main cause of climate change.But the program’s future is unclear. The federal Environmental Protection Agency has announced an agency reorganization plan under which its Energy Star offices would be eliminated.That would make it more difficult...",No URL Available,en,2025-06-25T23:58:53Z,suntimes,How to find energy-efficient appliances if Energy Star program ends,https://chicago.suntimes.com/consumer-affairs/2025/06/25/energy-star-energy-efficient-appliances-program-alternatives-tips,2025-06-25,23
Billy Heyen,sports,us,The new Boston defenseman could stick around.,https://library.sportingnews.com/styles/crop_style_16_9_desktop_webp/s3/2025-06/USATSI_25920677.jpg.webp?itok=eq8FuL87,en,2025-06-25T23:58:42Z,Sporting News,Bruins' Henri Jokiharju gets $10.8 million contract update,https://www.sportingnews.com/us/nhl/news/bruins-henri-jokiharju-108-million-contract-update/b7d54441b6fd3620af52ac14,2025-06-25,23
Claire Fahy and Alyce McFadden,general,us,The state assemblyman ran a campaign tightly focused on issues related to affordability. Here is a look at where he stands on those issues and others.,https://static01.nyt.com/images/2025/06/25/multimedia/25nyc-election-mamdani-on-the-issues-topart-zblf/25nyc-election-mamdani-on-the-issues-topart-zblf-mediumSquareAt3X.jpg,en,2025-06-25T23:58:26Z,The New York Times,A Look at Zohran Mamdani’s Stances on Key Issues,https://www.nytimes.com/2025/06/25/nyregion/mamdani-policies-stances.html,2025-06-25,23
Patrick Saunders,general,us,The lefty has succeeded at Coors Field but rarely dominated.,https://www.denverpost.com/wp-content/uploads/2017/04/clayton-kershaw.jpg?w=1400px&strip=all,en,2025-06-25T23:57:56Z,photos,"Dodgers’ Clayton Kershaw, eight strikeouts shy of 3,000, faces Rockies Thursday at Coors Field",https://www.denverpost.com/2025/06/25/clayton-kershaw-strikeouts-milestone-rockies-dodgers/,2025-06-25,23
Óscar Contreras,general,us,"Lakewood police is asking for the public’s help to find two suspects accused of stealing approximately $15,000 worth of tools and equipment out of The Academy Place last month.",No URL Available,en,2025-06-25T23:56:53Z,thedenverchannel,Lakewood police seeking public’s help to identify theft suspects accused of stealing $15k worth of equipment,https://www.denver7.com/news/crime/lakewood-police-seeking-publics-help-to-identify-theft-suspects-accused-of-stealing-15k-worth-of-equipment,2025-06-25,23
Investing.com,business,us,Lockheed Martin declares third quarter 2025 dividend of $3.30,https://i-invdn-com.investing.com/news/news_headline

In [0]:
# write to the gold adls
try:
    gold_write_path = f"{gold_path}/{current_date}"
    news.write\
        .format("parquet")\
            .mode("overwrite")\
                .save(gold_write_path)
    print(f"Data written to {gold_write_path} successfully.")
except Exception as e:
    print(f"Error writing data to {gold_write_path}: {e}")

Data written to abfss://gold@adlsnews.dfs.core.windows.net//2025-06-25 successfully.
